# Building an inverted index

  - You are given a sample (1000 documents) from the [The Reuters-21578 data collection](http://www.daviddlewis.com/resources/testcollections/reuters21578/) in `data/reuters21578-000.xml`
  - The code that parses the XML and extract a list of preprocessed terms (tokenized, lowercased, stopwords removed) is already given
  - You are also given an `InvertedIndex` class that manages the posting lists operations
  - Your task is to build an inverted index from the input collection.

In [ ]:
import ipytest
import re

from collections import Counter
from xml.dom import minidom

ipytest.autoconfig()

## Parsing documents

Stopwords list

In [ ]:
STOPWORDS = ['a', 'an', 'and', 'are', 'as', 'at', 'be', 'but', 'by', 'for', 'if', 'in', 'into', 'is', 'it', 'no', 'not', 'of', 'on', 'or', 'such', 'that', 'the', 'their', 'then', 'there', 'these', 'they', 'this', 'to', 'was', 'will', 'with']

Stripping tags inside <> using regex

In [ ]:
def striptags(text):
    p = re.compile(r'<.*?>')
    return p.sub('', text)

Parse input text and return a list of indexable terms

In [ ]:
def parse(text):
    terms = []
    # Replace specific characters with space
    chars = ['\'', '.', ':', ',', '!', '?', '(', ')']
    for ch in chars:
        if ch in text:
            text = text.replace(ch, ' ')

    # Remove tags
    text = striptags(text)

    # Tokenization
    for term in text.split():  # default behavior of the split is to split on one or more whitespaces
        # Lowercasing
        term = term.lower()
        # Stopword removal
        if term in STOPWORDS:
            continue
        terms.append(term)

    return terms

## Processing the input document collection

  - The collection is given as a single XML file. 
  - Each document is inside `<REUTERS ...> </REUTERS>`.
  - We extract the contents of the `<DATE>`, `<TITLE>`, and `<BODY>` tags.
  - After each extracted document, the provided callback function is called and all document data is passed in a single dict argument.

In [ ]:
def process_collection(input_file, callback):
    xmldoc = minidom.parse(input_file)
    # Iterate documents in the XML file
    itemlist = xmldoc.getElementsByTagName('REUTERS')
    doc_id = 0
    for doc in itemlist:
        doc_id += 1
        date = doc.getElementsByTagName('DATE')[0].firstChild.nodeValue
        # Skip documents without a title or body
        if not (doc.getElementsByTagName('TITLE') and doc.getElementsByTagName('BODY')):
            continue
        title = doc.getElementsByTagName('TITLE')[0].firstChild.nodeValue
        body = doc.getElementsByTagName('BODY')[0].firstChild.nodeValue
        callback({
            'doc_id': doc_id,
            'date': date,
            'title': title,
            'body': body
            })

Prints a document's contents (used as a callback function passed to `process_collection`)

In [ ]:
def print_doc(doc):
    if doc['doc_id'] <= 5:  # print only the first 5 documents
        print('docID:', doc['doc_id'])
        print('date:', doc['date'])
        print('title:', doc['title'])
        print('body:', doc['body'])
        print('--')

In [ ]:
process_collection('data/reuters21578-000.xml', print_doc)

## Task 1: Complete the inverted index class

  - The inverted index is an object with methods for adding and fetching postings.
  - The data is stored in a map, where keys are terms and values are lists of postings.
  - Each posting is an object that holds the doc_id and an optional payload.

In [ ]:
class Posting(object):
    
    def __init__(self, doc_id, payload=None):
        self.doc_id = doc_id
        self.payload = payload
    
    def get_doc_id(self):
        return self.doc_id
    
    def get_payload(self):
        return self.payload

In [ ]:
class InvertedIndex(object):

    def __init__(self):
        self.index = {}
    
    def add_posting(self, term, doc_id, payload=None):
        """Adds a document to the posting list of a term."""
        # TODO: if term not in index, initialize empty posting list
        # TODO: append new posting to the posting list

    def get_postings(self, term):
        """Fetches the posting list for a given term."""
        # TODO: complete
        return None

    def get_terms(self):
        """Returns all unique terms in the index."""
        return self.index.keys() 

Tests.

In [ ]:
%%run_pytest[clean]

def test_postings():
    ind = InvertedIndex()
    ind.add_posting('term', 1, 1)
    ind.add_posting('term', 2, 4)
    # Testing existing term
    postings = ind.get_postings('term')
    assert len(postings) == 2
    assert postings[0].get_doc_id() == 1
    assert postings[0].get_payload() == 1
    assert postings[1].get_doc_id() == 2
    assert postings[1].get_payload() == 4
    # Testing non-existent term
    assert ind.get_postings("xyx") is None

def test_vocabulary():
    ind = InvertedIndex()
    ind.add_posting('term1', 1)
    ind.add_posting('term2', 1)
    ind.add_posting('term3', 2)
    ind.add_posting('term2', 3)
    assert set(ind.get_terms()) == set(['term1', 'term2', 'term3'])

## Task 2: Build an inverted index from the input collection

**TODO**: Complete the code to index the entire document collection.  (The content for each document should be the title and body concatenated)

In [ ]:
ind = InvertedIndex()

def index_doc(doc):
    text = doc['title'] + ' ' + doc['body']
    terms = parse(text)  # list of terms in the document    
    # TODO: index the document (add all terms with freqs using `ind.add_posting()`)
    
process_collection('data/reuters21578-000.xml', index_doc)

## Task 3: Save the inverted index to a file

Save the inverted index to a file (`data/index.dat`). Use a simple text format with `termID docID1:freq1 docID2:freq2 ...` per line, e.g.,

```
xxx 1:1 2:1 3:2
yyy 2:1 4:2
zzz 1:3 3:1 5:2
...
```

Implement this by (1) adding a `write_to_file(self, filename)` method to the `InvertedIndex` class and then (2) invoking that method in the cell below.

In [ ]:
# TODO

## Task 4 (advanced, optional): Plot collection size against index size

Create a plot that compares the size of the document collection (bytes) against the size of the corresponding index (bytes) on the y-axis vs. with respect to the number of documents on the x-axis. You may use [Matplotlib](https://www.tutorialspoint.com/jupyter/jupyter_notebook_plotting.htm) for plotting.

In [ ]:
# TODO

## Feedback

Please give (anonymous) feedback on this exercise by filling out [this form](https://forms.gle/2jPayczbFhEcC9K68).